# Homework 1: Data-driven decison making

Read the case study *Select Collections, Inc.* included with this assignment. You will play a role similar to that of the summer intern, Marcos Kilduff. However, your assignment is not to predict `totalpay` for the observations in the test set. Instead, your task is to **decide** which of the accounts in the test set should be purchased and to **evaluate** the profits that your purchasing decisions could generate. Purchasing any individual account costs 25% of the account's balance (`cobal`).

In [98]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
!gdown https://drive.google.com/uc?id=1Z66wSxIkkeJJoN-REG-D0_N2j8zmC52M
!gdown https://drive.google.com/uc?id=1S0G2OXd6V6RrkysAuq2XvkUAWzVNpaM3

df_train = pd.read_csv("SC_Training_Data.csv")
df_test = pd.read_csv("SC_Test_Data.csv") #no tatalpay
nickname = "ThreeThree"

Downloading...
From: https://drive.google.com/uc?id=1Z66wSxIkkeJJoN-REG-D0_N2j8zmC52M
To: /content/SC_Training_Data.csv
100% 245k/245k [00:00<00:00, 72.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1S0G2OXd6V6RrkysAuq2XvkUAWzVNpaM3
To: /content/SC_Test_Data.csv
100% 232k/232k [00:00<00:00, 79.0MB/s]


# Overview and preparing



In [87]:
# To see if the profit are correlated to states
data = df_train.copy()
data['profit'] = data['totalpay']-0.25*data['cobal']
data.groupby('state')['profit'].mean()

state
AK   -271.400000
AL   -199.691860
AR   -322.086207
AZ    -57.683544
CA     69.969375
CO     67.463235
CT    -62.483333
DC   -382.444444
DE   -550.450000
FL    -86.617965
GA    -76.473333
HI    -87.037500
IA   -311.130952
ID   -752.272727
IL     27.594828
IN    415.325000
KS   -298.509615
KY    235.943878
LA    183.088095
MA    289.446429
MD    -40.348485
ME   -168.350000
MI    252.246667
MN    234.375000
MO   -197.468750
MS    -91.177419
MT   -306.500000
NC     75.447368
ND   -744.750000
NE   -386.833333
NH   -644.833333
NJ    179.147500
NM   -216.375000
NV   -389.331633
NY     77.790640
OH     83.765152
OK     32.524194
OR    -48.493902
PA    -43.418067
RI    -69.269231
SC   -146.601695
SD   -767.972222
TN    316.250000
TX   -293.115091
UT   -599.062500
VA    167.754902
VT    300.062500
WA    302.533708
WI   -342.637097
WV   -579.361111
WY    602.406250
Name: profit, dtype: float64

In [88]:
# To see if the profit are correlated to rollout
data.groupby('rollout')['profit'].mean()

rollout
Associates    -79.700730
Bank_Of_Am    134.572108
Chase         -24.575893
Chase_Bony   -244.271739
Chase_Rev    -139.691624
Discover       -0.083616
Wells          17.111156
Wells_FIB    -240.970930
Name: profit, dtype: float64

In [99]:
# get dummies first and calculate the profit as y
df_train = pd.get_dummies(df_train, columns=['state','rollout'], drop_first=True) 
df_train['profit'] = df_train['totalpay']-0.25*df_train['cobal']
df_train.drop(columns='totalpay', inplace=True)

X_train = df_train.iloc[:,:-2]
y_train = df_train.iloc[:,-1]

#y = y_train>0
y = pd.DataFrame(y_train)
y['judge'] = y_train>0
X = X_train.drop(columns=['accessscr', 'acctid']) # highly correlate to lnaccessscr

# Classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Tree induction
from sklearn.linear_model import LogisticRegression #
from sklearn.svm import SVC #support vector machine

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# Algorithms
Tree_algorithm = DecisionTreeClassifier() 
SVM = SVC()
logreg = LogisticRegression()

# Grids
tree_grid = {"min_samples_leaf": [2, 4, 8, 16, 32, 64], 
             'max_features': ['auto', 'sqrt', 'log2'],
              'ccp_alpha': [0.1, .01, .001],
              'max_depth' : [5, 6, 7, 8, 9],
              'criterion' :['gini', 'entropy'], 
             'random_state': [0]
             }
svc_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100], 'max_iter': [-1]}, 
                               {'kernel': ['linear'], 'C': [1, 10, 100], 'max_iter': [-1]}]
log_grid={"C":np.logspace(-3,3,7)}

# Conduct grid searches, since the units of money lost or earned are the same, the costs of FP&FN are considered the same
tree_searcher = GridSearchCV(Tree_algorithm, tree_grid, cv=5, scoring = 'accuracy')
SVC = GridSearchCV(SVM, svc_grid, cv=5, scoring = 'accuracy') # running time too long, drop this method
logreg_cv=GridSearchCV(logreg, log_grid, cv=5, scoring = 'accuracy')

# Report results
best_algorithm_C = None
best_score = None
for searcher in [tree_searcher, logreg_cv]:
    searcher.fit(X, y['judge'])
    print(f"Algorithm : {type(searcher.best_estimator_)}")
    print(f"Best parameters: {searcher.best_params_}")
    print(f"Best score (negative custom error): {searcher.best_score_:.2f}")
    print(f"Profit: {y['profit'][searcher.best_estimator_.predict(X)>0].sum():.2f}")
    print("--------")
    if best_algorithm_C is None or best_score < searcher.best_score_:
        best_algorithm_C = searcher.best_estimator_
        best_paras = searcher.best_params_
        best_score = searcher.best_score_

Algorithm : <class 'sklearn.tree._classes.DecisionTreeClassifier'>
Best parameters: {'ccp_alpha': 0.001, 'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 32, 'random_state': 0}
Best score (negative custom error): 0.58
Profit: 37818.75
--------
Algorithm : <class 'sklearn.linear_model._logistic.LogisticRegression'>
Best parameters: {'C': 0.001}
Best score (negative custom error): 0.58
Profit: 58789.50
--------


# Regression

In [ ]:
X_train.head()

,acctid,zip,cobal,collscr,cs,accessscr,lnacscr,bureauscr,eaglemod,numcalls,...,state_WA,state_WI,state_WV,state_WY,rollout_Bank_Of_Am,rollout_Chase,rollout_Chase_Bony,rollout_Chase_Rev,rollout_Discover,rollout_Wells
0,1,93313,1786,4013181,4,0.1992,-1.61,133,67,104,...,0,0,0,0,0,0,0,0,0,1
1,2,92154,2048,1015805,1,0.4376,-0.83,109,6,233,...,0,0,0,0,0,0,0,0,0,1
2,3,33319,5170,4013057,4,0.8140,-0.21,103,90,28,...,0,0,0,0,0,0,0,0,0,1
3,4,98115,4506,8012265,8,0.2978,-1.21,136,11,8,...,1,0,0,0,1,0,0,0,0,0
4,5,49615,2706,3013630,3,0.3722,-0.99,105,42,45,...,0,0,0,0,0,0,0,0,0,0


In [100]:
from pandas.core.common import random_state
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

# Algorithms
ridge_algorithm = Ridge()
lasso_algorithm = Lasso()
tree_algorithm = DecisionTreeRegressor()

# Grids
lin_grid = {"alpha": [10**i for i in np.arange(0.9, 5, 0.1)]}
tree_grid = {"min_samples_leaf": range(1, 67, 3), 
             "criterion":['squared_error', 'absolute_error', 'friedman_mse']#
             }

# Conduct grid searches
tree_searcher = GridSearchCV(tree_algorithm, tree_grid, cv=5, scoring = 'r2')
ridge_searcher = GridSearchCV(ridge_algorithm, lin_grid, cv=5, scoring = 'r2')
lasso_searcher = GridSearchCV(lasso_algorithm, lin_grid, cv=5, scoring = 'r2')

# Report results
best_algorithm = None
best_score = None
for searcher in [tree_searcher, ridge_searcher, lasso_searcher]: 
    searcher.fit(X_train, y_train)
    print(f"Algorithm : {type(searcher.best_estimator_)}")
    print(f"Best parameters: {searcher.best_params_}")
    print(f"Best score: {searcher.best_score_:.2f}")
    print(f"Total Profit: {y_train[searcher.best_estimator_.predict(X_train)>0].sum()}")
    print("--------")
    if best_algorithm is None or best_score < searcher.best_score_:
        best_algorithm = searcher.best_estimator_
        best_paras = searcher.best_params_
        best_score = searcher.best_score_

Algorithm : <class 'sklearn.tree._classes.DecisionTreeRegressor'>
Best parameters: {'criterion': 'squared_error', 'min_samples_leaf': 55}
Best score: 0.05
Total Profit: 555232.75
--------
Algorithm : <class 'sklearn.linear_model._ridge.Ridge'>
Best parameters: {'alpha': 31.622776601683793}
Best score: 0.09
Total Profit: 456597.75
--------
Algorithm : <class 'sklearn.linear_model._coordinate_descent.Lasso'>
Best parameters: {'alpha': 7.943282347242816}
Best score: 0.08
Total Profit: 410145.5
--------


In [ ]:
y_train

0        -86.50
1       -212.00
2      -1282.50
3       1423.50
4       1323.50
         ...   
3565    1024.75
3566    -390.75
3567    -692.75
3568     197.25
3569     541.00
Name: profit, Length: 3570, dtype: float64

In [101]:
import statsmodels.api as sm
model_L = sm.OLS(y_train,sm.add_constant(X_train.drop(columns = ['accessscr', 'acctid'])))
results = model_L.fit()
print(results.summary())
amount = results.predict(sm.add_constant(X_train.drop(columns = ['accessscr', 'acctid'])))
print(y_train[amount > 0].sum(), y_train[amount > 0].sum()/y_train[y['judge']].sum())

                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.122
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     7.457
Date:                Wed, 23 Nov 2022   Prob (F-statistic):           1.87e-60
Time:                        12:57:11   Log-Likelihood:                -29526.
No. Observations:                3570   AIC:                         5.918e+04
Df Residuals:                    3504   BIC:                         5.959e+04
Df Model:                          65                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               -739.2549   1435

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [102]:
# drop some insignificant dummy variables since the coefficients of these variables are large
model_L = sm.OLS(y_train,sm.add_constant(X_train[['acctid', 'zip', 'cobal', 'cs', 'lnacscr', 
                                                  'bureauscr', 'eaglemod', 'numcalls', 'numrpcs', 'state_WY', 'rollout_Bank_Of_Am']]))
results2 = model_L.fit()
print(results2.summary())
amount2 = results2.predict(sm.add_constant(X_train[['acctid', 'zip', 'cobal', 'cs', 'lnacscr', 
                                                  'bureauscr', 'eaglemod', 'numcalls', 'numrpcs', 'state_WY', 'rollout_Bank_Of_Am']]))
print(y_train[amount2 > 0].sum(), y_train[amount2 > 0].sum()/y_train[y['judge']].sum())

                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     33.17
Date:                Wed, 23 Nov 2022   Prob (F-statistic):           7.30e-68
Time:                        12:57:34   Log-Likelihood:                -29583.
No. Observations:                3570   AIC:                         5.919e+04
Df Residuals:                    3558   BIC:                         5.926e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                653.3386    139

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


# Prediction

In [104]:
df_test = pd.get_dummies(df_test, columns=['state','rollout'], drop_first=True) 

In [ ]:
df_test.drop(columns=X.columns)

,accessscr,rollout_Wells_FIB
0,0.4030,0
1,0.1662,0
2,0.6252,0
3,0.4670,0
4,0.7620,0
...,...,...
3565,0.5990,0
3566,0.6573,0
3567,0.5458,0
3568,0.3933,0


In [110]:
'''
we use Ridge Regression to predict the results
since the R-square of Ridge Regression method is larger than Lasso and Decision Tree regression and 
the Total Profit is the largest among all classification methods and regression methods we have tried 
'''
pred = pd.Series(best_algorithm.predict(df_test.loc[:,X_train.columns]), index = df_test.index)

In [111]:
decisions = (pred>0)+0
df_decisions = pd.DataFrame({"acctid":df_test["acctid"], "decision": decisions})
df_decisions.to_csv(f"decisions_{nickname}.csv", index=False)

In [112]:
estimate = pred[pred>0].sum()
estimate = int(estimate)
with open(f"estimate_{nickname}.txt", "w") as text_file:
    text_file.write(f"{estimate:.0f}")